#2. 데이터 확인

> 위와 같이 영화별로 평점을 처리한 데이터가 존재한다. 이를 이용하여 협업 필터링을 이용한 추천 시스템을 만들어 보자

#2. 데이터 가공 및 변환

##2.1 데이터 가공

>위와 같이 row는 영화 이름을 columns은 사용자 id를 갖는 dataFrame을 만들 수 있다.

##2.2 데이터 변환
>위와 같이 가공한 데이터에서 평가가 되지 않는 부분에 대해서는 결측치로 나타나게 된다. 이를 해결하기 위해 결측치를 0으로 대처하자

>NaN으로 값이 지정되어 있으면 유사도를 구할 수 없기 때문에 0으로 변환시켰다.

#3. 영화 간 유사도 산출

##3.1 사용자 기반 추천
> 유사도 산출을 위해 cosine_similarity()를 이용할 것이다. cosine_similarity()는 행을 기준으로 유사도를 산출하는 함수이다. 현재 dataframe의 행은 사용자ID로 이루어져 있어 유사도를 산출하게 되면 사용자의 유사도를 구할 수 있다.

>위와 같이 코사인 유사도를 이용하여 사용자별 평점 유사도를 측정할 수 있다.

>이를 간단하게 DataFrame으로 만들고 user1과 유사도가 높은 순으로 정렬하여 출력했다. user1과 영화적 취향이 가장 비슷한 사람은 'user2'이다. 따라서 user1에게 user2가 본 영화를 추천하면 좋을 것이다.

> user2와 영화적 취향이 비슷하다고 판단하여 user2가 본 영화의 정렬된 정보를 확인했다. 이를 간단하게 함수를 만들어 보자.

>위와 같이 영화적 취향이 비슷한 사람의 영화를 보여주고 있다. 이번에는 사용자가 못본 영화를 추천하는 시스템을 만들어 보자.

>안본 영화의 평점을 0으로 수정했음으로 사용자의 평점이 0과 같은 정보를 추출하면 된다. 하지만 현재 columns 정보가 영화 정보로 되어 있어 정보를 비교할 수 없다. transpose가 전치를 해 주는 함수이므로 이를 이용하여 치환하고 평점이 0인 값만 출력했다.

> 이렇게 구한 테이블에서 사용자와 취향이 비슷한 사용자의 정보 중 보지 않은 정보를 추천할 수도 있다. 이를 간단하게 함수로 만들어 보자.

##3.2 아이템 기반 추천
> 기존의 DataFrame은 사용자를 행으로 가지고 있다. 아이템 기반 추천을 위해 행을 영화 제목으로 변경하고 시스템을 구축해 보자.

>위와 같이 transpose() 함수를 이용하여 간단하게 행, 열을 치환할 수 있다.

>각 영화별 평점 유사도를 구한 결과이다. 

>각 영화별 평점에 따른 유사도를 dataframe으로 표현해 봤다.

> 7번방의 선물과 연관성이 높은 영화 순으로 나열해 보니 연평대전을 추천해 주고 있다. [1:]을 이용하여 자기 자신을 제외하고 영화를 추천해 주었다.
